In [3]:

# importing required libraries
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np


# nltk libraries used for NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# from sklearn.naive_bayes import MultinomialNB


In [4]:
df = pd.read_csv('../dataset/train_cleaned3.csv')


In [5]:
df.head()

,id,name,document_text,cat_name
0,22474,"[ 'inform ' , 'regard ' , 'merger ' , 'navio '...","[ 'at ' , 'special ' , 'meet ' , 'held ' , 'ma...",Corporate Communications
1,27460,"[ 'announc ' , 'approv ' , 'chang ' , 'members...","[ 'on ' , 'april ' , ' 2 ' , '2021 ' , 'china ...",Securities Settlement
2,6926,"[ 'sfc ' , 'suspend ' , 'shiu ' , 'yau ' , 'wa...","[ 'the ' , 'secur ' , 'futur ' , 'commiss ' , ...",Antitrust
3,6982,"[ 'renminbi ' , 'rmb ' , 'haircut ' , 'februar...","[ 'pursuant ' , 'section ' , '262 ' , 'clear '...",Securities Settlement
4,5022,"[ 'antimoney ' , 'launder ' , 'counter ' , 'fi...","[ 'money ' , 'launder ' , 'terror ' , 'financ ...",Financial Crime


In [113]:
df['name'] = df['name'] + df['document_text']

In [116]:
X = df[['name']]
y = df['cat_name']

In [117]:
X.head()

,name
0,"[ 'inform ' , 'regard ' , 'merger ' , 'navio '..."
1,"[ 'announc ' , 'approv ' , 'chang ' , 'members..."
2,"[ 'sfc ' , 'suspend ' , 'shiu ' , 'yau ' , 'wa..."
3,"[ 'renminbi ' , 'rmb ' , 'haircut ' , 'februar..."
4,"[ 'antimoney ' , 'launder ' , 'counter ' , 'fi..."


In [118]:
y.head()

0    Corporate Communications
1       Securities Settlement
2                   Antitrust
3       Securities Settlement
4             Financial Crime
Name: cat_name, dtype: object

In [133]:
# Label encoding on target variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

array([ 8, 47,  1, ..., 18, 13, 14])

In [119]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size = 0.25, random_state = 21)
print(X_train.shape, y_train.shape)
print(X_cv.shape, y_cv.shape)

(35326, 1) (35326,)
(11776, 1) (11776,)


In [120]:
combined_vocab = X_train['name'].tolist() + X_cv['name'].tolist()

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df = 10, max_features = 1000, ngram_range = (1, 3))
tfidf.fit(combined_vocab)

TfidfVectorizer(max_features=1000, min_df=10, ngram_range=(1, 3))

In [123]:
X_train_tfidf = tfidf.transform(X_train['name'])
X_cv_tfidf = tfidf.transform(X_cv['name'])


In [125]:
print(X_train_tfidf.shape, y_train.shape)
print(X_cv_tfidf.shape, y_cv.shape)

(35326, 1000) (35326,)
(11776, 1000) (11776,)


In [132]:
# y_cv

In [126]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [127]:
# instantiate OneVsRestClassifier
clf = OneVsRestClassifier(LogisticRegression())
clf.fit(X_train_tfidf, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [128]:
prediction = clf.predict(X_cv_tfidf)

In [129]:
prediction

array(['Trade Pricing', 'Corporate Governance', 'Securities Clearing',
       ..., 'Forms', 'Legal Proceedings', 'Securities Sales'],
      dtype='<U40')

In [135]:
df.head()

,id,name,document_text,cat_name,cat_len
0,22474,"[ 'inform ' , 'regard ' , 'merger ' , 'navio '...","[ 'at ' , 'special ' , 'meet ' , 'held ' , 'ma...",Corporate Communications,2
1,27460,"[ 'announc ' , 'approv ' , 'chang ' , 'members...","[ 'on ' , 'april ' , ' 2 ' , '2021 ' , 'china ...",Securities Settlement,2
2,6926,"[ 'sfc ' , 'suspend ' , 'shiu ' , 'yau ' , 'wa...","[ 'the ' , 'secur ' , 'futur ' , 'commiss ' , ...",Antitrust,1
3,6982,"[ 'renminbi ' , 'rmb ' , 'haircut ' , 'februar...","[ 'pursuant ' , 'section ' , '262 ' , 'clear '...",Securities Settlement,2
4,5022,"[ 'antimoney ' , 'launder ' , 'counter ' , 'fi...","[ 'money ' , 'launder ' , 'terror ' , 'financ ...",Financial Crime,2


In [136]:
final_result_with_id_df = pd.DataFrame(df['id'],prediction)

In [139]:
y_cv

0        22474
1        27460
2         6926
3         6982
4         5022
         ...  
47097    14825
47098     6875
47099     8901
47100     6285
47101    16675
Name: id, Length: 47102, dtype: int64

In [144]:
index = list(y_cv.index)

In [147]:
d = pd.DataFrame({'id':index,'y_test':list(y_cv),'prediction':list(prediction)})

In [155]:
d.head(50)

,id,y_test,prediction
0,1041,Securities Sales,Trade Pricing
1,26450,Fees and Charges,Corporate Governance
2,8478,Market Risk,Securities Clearing
3,43967,Compliance Management,Exemptions
4,19419,Market Risk,Compliance Management
5,38357,Payments and Settlements,Regulatory Actions
6,26691,Natural Disasters,Powers and Duties
7,19570,Monetary and Economic Policy,Natural Disasters
8,9369,Regulatory Actions,Money Services
9,1524,Fees and Charges,Fees and Charges


ValueError: Item wrong length 1 instead of 47102.